In [1]:
import numpy as np
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup as bs
#from google.colab import files
from tqdm import *
#courses = pd.read_csv('courses_all.csv')
#career360_courses = pd.DataFrame(columns=['College','City','State','Course','Duration','Study mode','Approved intake','Course level','Course Fee','Approval','Eligiblity','Course_Detail'])

In [2]:
eng_list = []

In [3]:
for i in tqdm(range(20)):
    page = rq.get('https://media.careers360.com/colleges/list-of-media-journalism-colleges-in-India?page='+str(i))
    soup = bs(page.content,'html.parser')
    vals = soup.find_all('div',class_='title')
    for val in vals:
        try:
            eng_list.append(val.a['href'])
        except:
            pass

100%|██████████| 20/20 [00:22<00:00,  1.15s/it]


In [4]:
courses = pd.DataFrame(columns=[0,1,2])

In [5]:
idx = 0
for link in tqdm(eng_list):
    try:
        page = rq.get(link)
        soup = bs(page.content,'html.parser')
        courses.loc[idx]=[link,soup.find_all('h4',class_='blockSubHeading')[0].get_text()[-2:],'https://media.careers360.com/'+soup.find_all('ul',class_='list-tabs')[0].find_all('li')[1].a['href']]
        idx+=1
    except:
        pass

100%|██████████| 594/594 [05:39<00:00,  1.75it/s]


In [6]:
courses.to_csv('media_courses.csv')

In [7]:
from threading import Thread
def scrapper(start, end):
    career360_courses = pd.DataFrame(columns=['College','City','State','Course','Duration','Study mode','Approved intake','Course level','Course Fee','Approval','Eligiblity','Course_Detail'])
    indx = 0
    cnt = start - 1
    for header_course in tqdm(courses[2][start:end]):
        cnt += 1
        try:
            num = int(np.ceil(int(courses[1][cnt])/10))
        except:
            continue
    #     print(f'num = {num}')
        for page_num in tqdm(range(num)):
            link = header_course + '?levelId=all&courseNid=all-all&page='+str(page_num)
    #         print(link)
            page = rq.get(link)
            course_soup = bs(page.content, 'html.parser')
        for course_link in course_soup.find_all('span',class_='readmore'):
            course_link = 'https://media.careers360.com/' + course_link.a['href']
            #             print(f'course_link = {course_link}')
            page1 = rq.get(course_link)
            soup = bs(page1.content,'html.parser')
            career360_courses.loc[indx]=None

            try:
                career360_courses.loc[indx]['Course'] = soup.find_all('h2',class_='block-title')[0].get_text()
                career360_courses.loc[indx]['College']=soup.find_all('h1',class_='titleNameCol')[0].get_text()
                career360_courses.loc[indx]['City'] = soup.find_all('ul',class_='clg-info')[0].find_all('li')[0].find_all('a')[0].get_text()
                career360_courses.loc[indx]['State'] = soup.find_all('ul',class_='clg-info')[0].find_all('li')[0].find_all('a')[1].get_text()
            except:
                pass

            try:
                career360_courses.loc[indx]['Eligiblity'] = soup.find_all('span',class_='more-eligibility')[0].get_text()[:-11]
            except:
                try:
                    career360_courses.loc[indx]['Eligiblity'] = soup.find_all('div',class_='default-elig')[0].get_text()
                except:
                    career360_courses.loc[indx]['Eligiblity'] = None

            try:
                career360_courses.loc[indx]['Course_Detail'] = (soup.find_all('span',class_='moreCourseDetails')[0].p.get_text())[:-11]
            except:
                try:
                    career360_courses.loc[indx]['Course_Detail'] = (soup.find_all('div',class_='default-course-details')[0].get_text())
                except:
                    career360_courses.loc[indx]['Course_Detail'] = None

            for row in soup.find_all('div',class_='coursesPageLableInnerSec'):
                career360_courses.loc[indx][row.strong.get_text()] = row.p.get_text()
            indx+=1
    career360_courses.to_csv(f'career360_scrap_dm_law({start}-{end}).csv')

In [8]:
siz = len(eng_list)
numer = int(siz/20)
prev = 0
def threader():
    threads = []
    for i in range(20):
        if i!=19:
            thread = Thread(target=scrapper, args = (i*numer,(i+1)*numer))
            threads.append(thread)
            thread.start()
        else:
            thread = Thread(target=scrapper, args = (19*numer,siz))
            threads.append(thread)
            thread.start()        
    
    for thread in threads:  # iterates over the threads
        thread.join()
                    
    print('done')

In [9]:
threader()

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]


  0%|          | 0/1 [00:00<?, ?it/s]



  0%|          | 0/29 [00:00<?, ?it/s]




  0%|          | 0/1 [00:00<?, ?it/s]





  0%|          | 0/29 [00:00<?, ?it/s]






  0%|          | 0/1 [00:00<?, ?it/s]







  0%|          | 0/29 [00:00<?, ?it/s]








  0%|          | 0/1 [00:00<?, ?it/s]









  0%|          | 0/29 [00:00<?, ?it/s]










  0%|          | 0/1 [00:00<?, ?it/s]











  0%|          | 0/29 [00:00<?, ?it/s]












  0%|          | 0/1 [00:00<?, ?it/s]













  0%|          | 0/29 [00:00<?, ?it/s]














  0%|          | 0/1 [00:00<?, ?it/s]















  0%|          | 0/29 [00:00<?, ?it/s]
















  0%|          | 0/1 [00:00<?, ?it/s]

















  0%|          | 0/29 [00:00<?, ?it/s]


















  0%|          | 0/1 [00:00<?, ?it/s]



















  0%|          | 0/29 [00:00<?, ?it/s]




















  0%|          | 0/1

  3%|▎         | 1/29 [00:05<02:43,  5.84s/it]





















  0%|          | 0/1 [00:00<?, ?it/s]














  3%|▎         | 1/29 [00:05<02:42,  5.81s/it]






















  0%|          | 0/1 [00:00<?, ?it/s]




  3%|▎         | 1/29 [00:06<03:12,  6.86s/it]























  0%|          | 0/1 [00:00<?, ?it/s]

















  3%|▎         | 1/29 [00:07<03:19,  7.14s/it]
























  0%|          | 0/1 [00:00<?, ?it/s]



















100%|██████████| 1/1 [00:03<00:00,  3.03s/it]
































  3%|▎         | 1/29 [00:07<03:34,  7.67s/it]
























  0%|          | 0/1 [00:00<?, ?it/s]
















  3%|▎         | 1/29 [00:07<03:33,  7.62s/it]

























  0%|          | 0/1 [00:00<?, ?it/s]

  3%|▎         | 1/29 [00:08<03:44,  8.02s/it]


























  0%|          | 0/1 [00:00<?, ?it/s]













  3%|▎         | 1/39 [00:07<04:54,  7.75s/it]



























  0%

  0%|          | 0/1 [00:00<?, ?it/s]



















100%|██████████| 1/1 [00:02<00:00,  2.47s/it]





































  7%|▋         | 2/29 [00:13<03:04,  6.83s/it]


























  0%|          | 0/1 [00:00<?, ?it/s]




















100%|██████████| 1/1 [00:02<00:00,  2.53s/it]






















  7%|▋         | 2/29 [00:14<03:11,  7.11s/it]


























  0%|          | 0/1 [00:00<?, ?it/s]



















100%|██████████| 1/1 [00:03<00:00,  3.28s/it]








































100%|██████████| 1/1 [00:01<00:00,  1.89s/it]








































100%|██████████| 1/1 [00:03<00:00,  3.72s/it]



























 10%|█         | 3/29 [00:15<02:15,  5.20s/it]
























  0%|          | 0/1 [00:00<?, ?it/s]





















100%|██████████| 1/1 [00:02<00:00,  2.60s/it]

































  3%|▎         | 1/29 [00:15<07:21, 15.78s/it]
























  0%| 

  0%|          | 0/1 [00:00<?, ?it/s]











  7%|▋         | 2/29 [00:21<04:45, 10.58s/it]

























  0%|          | 0/1 [00:00<?, ?it/s]


















 10%|█         | 3/29 [00:21<03:04,  7.09s/it]


























  0%|          | 0/1 [00:00<?, ?it/s]




















100%|██████████| 1/1 [00:02<00:00,  2.39s/it]





































 10%|█         | 3/29 [00:21<03:10,  7.33s/it]


























  0%|          | 0/1 [00:00<?, ?it/s]




















100%|██████████| 1/1 [00:02<00:00,  2.52s/it]








































100%|██████████| 1/1 [00:02<00:00,  2.98s/it]









































100%|██████████| 1/1 [00:01<00:00,  1.83s/it]































 14%|█▍        | 4/29 [00:23<02:25,  5.80s/it]
























  0%|          | 0/1 [00:00<?, ?it/s]





















100%|██████████| 1/1 [00:02<00:00,  2.06s/it]









































100%|██████




















 14%|█▍        | 4/29 [00:27<02:52,  6.92s/it]


























  0%|          | 0/1 [00:00<?, ?it/s]



















100%|██████████| 1/1 [00:03<00:00,  3.19s/it]





























 17%|█▋        | 5/29 [00:28<02:17,  5.73s/it]


























  0%|          | 0/1 [00:00<?, ?it/s]























 17%|█▋        | 1/6 [00:01<00:08,  1.62s/it]



















100%|██████████| 1/1 [00:02<00:00,  2.75s/it]







































100%|██████████| 1/1 [00:02<00:00,  2.75s/it]







































100%|██████████| 1/1 [00:02<00:00,  2.89s/it]























 17%|█▋        | 5/29 [00:29<02:20,  5.85s/it]
























  0%|          | 0/1 [00:00<?, ?it/s]



















100%|██████████| 1/1 [00:02<00:00,  2.64s/it]








































 20%|██        | 1/5 [00:02<00:09,  2.38s/it]














 14%|█▍        | 4/29 [00:29<03:04,  7.39s/it]



















...








 79%|███████▉  | 23/29 [03:25<00:53,  8.95s/it]





















 83%|████████▎ | 24/29 [03:26<00:43,  8.62s/it]






















  0%|          | 0/1 [00:00<?, ?it/s]

















100%|██████████| 1/1 [00:02<00:00,  2.70s/it]





































100%|██████████| 1/1 [00:01<00:00,  1.80s/it]





































100%|██████████| 1/1 [00:01<00:00,  1.96s/it]



































100%|██████████| 1/1 [00:02<00:00,  2.40s/it]

























 93%|█████████▎| 27/29 [03:27<00:15,  7.69s/it]



















  0%|          | 0/1 [00:00<?, ?it/s]












 86%|████████▌ | 25/29 [03:27<00:33,  8.30s/it]




















  0%|          | 0/1 [00:00<?, ?it/s]













 67%|██████▋   | 26/39 [03:27<01:43,  7.99s/it]





















  0%|          | 0/1 [00:00<?, ?it/s]

 62%|██████▏   | 18/29 [03:28<02:07, 11.56s/it]






















  0%|          | 0/1 [00:00<?, ?it/s]


 86%|████████▌ | 25/29 [03:

100%|██████████| 1/1 [00:02<00:00,  2.52s/it]
























 72%|███████▏  | 28/39 [03:35<01:24,  7.70s/it]
















  0%|          | 0/1 [00:00<?, ?it/s]














100%|██████████| 1/1 [00:02<00:00,  2.15s/it]


























 83%|████████▎ | 24/29 [03:35<00:44,  9.00s/it]
















  0%|          | 0/1 [00:00<?, ?it/s]













100%|██████████| 1/1 [00:02<00:00,  2.65s/it]




















 86%|████████▌ | 25/29 [03:36<00:34,  8.66s/it]
















 90%|████████▉ | 26/29 [03:36<00:25,  8.34s/it]

















  0%|          | 0/1 [00:00<?, ?it/s]













100%|██████████| 1/1 [00:02<00:00,  2.38s/it]
















 93%|█████████▎| 27/29 [03:38<00:16,  8.08s/it]

















  0%|          | 0/1 [00:00<?, ?it/s]





 62%|██████▏   | 18/29 [03:38<02:13, 12.15s/it]


















  0%|          | 0/1 [00:00<?, ?it/s]







 86%|████████▌ | 25/29 [03:38<00:35,  8.75s/it]



















  0%|          | 0/1 [00:00<?, ?i














100%|██████████| 1/1 [00:02<00:00,  2.48s/it]

























100%|██████████| 1/1 [00:02<00:00,  2.44s/it]























100%|██████████| 1/1 [00:02<00:00,  2.83s/it]





















100%|██████████| 1/1 [00:02<00:00,  2.83s/it]











 97%|█████████▋| 28/29 [03:52<00:08,  8.30s/it]










  0%|          | 0/1 [00:00<?, ?it/s]








 79%|███████▉  | 31/39 [03:52<00:59,  7.49s/it]











  0%|          | 0/1 [00:00<?, ?it/s]


 97%|█████████▋| 28/29 [03:52<00:08,  8.30s/it]












  0%|          | 0/1 [00:00<?, ?it/s]










100%|██████████| 1/1 [00:01<00:00,  1.92s/it]






















100%|██████████| 1/1 [00:01<00:00,  1.92s/it]















 93%|█████████▎| 27/29 [03:54<00:17,  8.68s/it]











  0%|          | 0/1 [00:00<?, ?it/s]




 72%|███████▏  | 21/29 [03:54<01:29, 11.17s/it]












  0%|          | 0/1 [00:00<?, ?it/s]










100%|██████████| 1/1 [00:02<00:00,  2.17s/it]
















 97%|███████

100%|██████████| 29/29 [04:34<00:00,  9.46s/it]


 97%|█████████▋| 28/29 [04:36<00:09,  9.86s/it]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:01<00:00,  1.87s/it]



100%|██████████| 29/29 [04:39<00:00,  9.65s/it]

 52%|█████▏    | 15/29 [12:24<11:34, 49.63s/it]

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:02<00:00,  2.57s/it]


 55%|█████▌    | 16/29 [12:29<10:09, 46.87s/it]

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:02<00:00,  2.50s/it]


 59%|█████▊    | 17/29 [12:36<08:54, 44.51s/it]

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:02<00:00,  2.48s/it]


 62%|██████▏   | 18/29 [12:44<07:47, 42.46s/it]

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:02<00:00,  2.23s/it]


 66%|██████▌   | 19/29 [12:49<06:45, 40.52s/it]

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:02<00:00,  2.17s/it]


 69%|██████▉   | 20/29 [12:53<05:48, 38.68s/it]

  0%|          | 0/1 [00:00<?, ?i

done


In [10]:
scrapper(0,29)


  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:01<00:00,  1.46s/it]


  3%|▎         | 1/29 [00:04<01:52,  4.03s/it]

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:01<00:00,  1.13s/it]


  7%|▋         | 2/29 [00:08<01:55,  4.27s/it]

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:01<00:00,  1.03s/it]


 10%|█         | 3/29 [00:11<01:36,  3.71s/it]

  0%|          | 0/5 [00:00<?, ?it/s]

 20%|██        | 1/5 [00:01<00:04,  1.12s/it]

 40%|████      | 2/5 [00:02<00:03,  1.09s/it]

 60%|██████    | 3/5 [00:03<00:02,  1.08s/it]

 80%|████████  | 4/5 [00:03<00:00,  1.07it/s]

100%|██████████| 5/5 [00:04<00:00,  1.03it/s]


 14%|█▍        | 4/29 [00:15<01:39,  4.00s/it]

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:02<00:00,  2.38s/it]


 17%|█▋        | 5/29 [00:37<02:58,  7.42s/it]

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:02<00:00,  2.23s/it]


 21%|██  